In [10]:
import pandas as pd
import glob
import csv
import os

In [11]:
def get_description(maquina):
    df = pd.read_csv(maquina)
    return df.iloc[0]['Descrição']
        
def check_names(nome_lista, maquina_atual):
    return nome_lista in maquina_atual

def filter_name(nome_lista):
    return nome_lista.replace(";", "")

In [17]:
def listagem_maquinas():
    maquinas = glob.glob('maquinas/*.csv')
    nomes = pd.read_csv('nomes/nomes_maquinas.csv')
    with open('lista.csv', 'w') as csvfile:
        filewriter = csv.writer(
            csvfile, 
            lineterminator='\n',
            delimiter=',',
            quotechar='|', 
            quoting=csv.QUOTE_MINIMAL
        )
        filewriter.writerow(['Nome', 'Descricao'])

        for i in range(nomes.shape[0]):
            nome_puro = filter_name(nomes.iloc[i][0])

            for j in range(len(maquinas)):
                if check_names(nome_puro, maquinas[j]) == True:
                    descricao = get_description(maquinas[j])
                    filewriter.writerow([nome_puro,descricao])
                    break
            else:
                descricao = "nao exportado"
                filewriter.writerow([nome_puro,descricao])


In [42]:
def adiciona_novas_colunas():
    df = pd.read_csv('lista.csv')
    df = df.drop(['Descricao'], axis=1)
    df['Modelo'] = ''
    df['Setor'] = ''
    df['Sala'] = ''
    df.to_csv('nova_lista.csv')

In [43]:
adiciona_novas_colunas()